In [2]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
file_wheat = "/Volumes/toshiba/bio/IWGSC/by_chr/iwgsc_refseqv1.0_chr3A.fsa"

In [3]:
file_genes = "/Volumes/toshiba/bio/daniela/counts/svevo.counts.fa"
file_limit = "/Volumes/toshiba/bio/daniela/counts/svevo.limit.fa"
gene_list = ['TRITD3Av1G026860','TRITD3Av1G027040','TRITD3Av1G036950','TRITD3Av1G036910']

In [10]:
file_genes = "/Volumes/toshiba/bio/daniela/counts/zav_v2.counts.fa"
file_limit = "/Volumes/toshiba/bio/daniela/counts/zavitan.limit.fa"
gene_list = ['TRIDC3Av2G023690','TRIDC3Av2G024160','TRIDC3Av2G034660', 'TRIDC3Av2G034210', 'TRIDC3Av2G034600','TRIDC3Av2G034360']


In [11]:
fasta_file = SeqIO.parse(file_genes, 'fasta')
buffer_seqs = []
for record in fasta_file:
    if record.id in gene_list:
        buffer_seqs.append(record)
SeqIO.write(buffer_seqs, file_limit, "fasta")


6

In [ ]:
# cat disk/counts/svevo.limit.fa disk/counts/zavitan.limit.fa > disk/counts/limits.fa